In [ ]:
from dataclasses import dataclass
from collections import deque

@dataclass
class Node:
    name : str
    offset : list
    channels : dict
    child : list
    parent : str
    first_in : bool
    
file_path = "walk60.bvh"
bvh_data = []
with open(file_path, 'r') as f:
    for line in f:
        bvh_data.append(line.split())

num_frames = 0
frame_time = 0
frames = []
joint_List = []
info_List = []
for line in bvh_data:
    this_line = line
    if(this_line[0] == "Hierarchy"): continue
    if(this_line[0] == "MOTION"): continue
    if(this_line[0] == "Frames:"):
        num_frames = int(this_line[1])
        continue
    if(this_line[0] == "Frame"):
        frame_time = float(this_line[2])
        continue
    if(this_line[0] == "{" or
       this_line[0] == "ROOT" or
       this_line[0] == "}" or
       this_line[0] == "End" or
       this_line[0] == "JOINT" or
       this_line[0] == "OFFSET" or
       this_line[0] == "CHANNELS"):
        info_List.append(this_line)
        continue
    frames.append(this_line)
    
# # 处理栈中数据
# def Process_data(stack):
#     some_data = []
#     while(stack):
#         info_pop = stack.pop()
#         if(type(info_pop) != Node and info_pop[0] == "{"):
#             break
#         elif(type(info_pop) != Node and info_pop[0] == "}"):
#             continue
#         else:
#             some_data.append(info_pop)
#     print(some_data)
#     now_node = Node(
#         name = None,
#         offset = [],
#         channels = {},
#         child = [],
#         parent = None,
#         first_in = True
#     )
#     last_node = None
#     for i in some_data:
#         # if(type(i) != Node and i[0] == "OFFSET"):
#         #     now_node.offset = [float(i[1]), float(i[2]), float(i[3])]
#         # elif(type(i) != Node  and i[0] == "CHANNELS"):
#         #     now_node.channels = {
#         #         "num_channels" : int(i[1]),
#         #         "channels" : [i[1:]]
#         #     }
#         # elif(type(i) != Node and i[0] == "JOINT"):
#         #     now_node.name = i[1]
#         # elif(type(i) == Node):
#         #     last_node = i
#         #     i += 1
#         if(type(i) == Node):
#             last_node = i
#             continue
#         if(type(i) != Node and i[0] == "OFFSET"):
#             now_node.offset = [float(i[1]), float(i[2]), float(i[3])]
#         elif(type(i) != Node and i[0] == "CHANNELS"):
#             now_node.channels = {
#                 "num_channels" : int(i[1]),
#                 "channels" : [i[1:]]}
#         elif(type(i) != Node and i[0] == "End" or i[0] == "Joint"):
#             if(i[0] == "End") : last_node.name = i[0] + i[1]
#             if(i[0] == "Joint") : last_node.name = i[1]
#             last_node.parent = now_node.name
#             now_node.child.append(last_node)
#             last_node.first_in = False
#             joint_List.append(last_node)
            
#         print(now_node)
#         print(last_node)
#     return now_node
        
    
            
len_info = len(info_List)
node_list = []
stack = deque()
# for i in range(len_info):
#     if(info_List[i][0] == "{"):
#         stack.append(info_List[i])
#         while(info_List[i][0] != "}"):
#             i += 1
#             stack.append(info_List[i])
#     elif(info_List[i][0] == "}"):
#         stack.pop()
#         print(stack.pop())
for i in range(len_info):
    if info_List[i][0] == "ROOT":
        root = Node(
             name = info_List[i][0] + info_List[i][1],
                offset = [],
                channels = {},
                child = [],
                parent = None,
                first_in = True)
        node_list.append(root)
        joint_List.append(root)
        continue
    if info_List[i][0] == "{" or info_List[i][0] == "OFFSET" or \
        info_List[i][0] == "CHANNELS" or info_List[i][0] == "End" or \
        info_List[i][0] == "JOINT":
            stack.append(info_List[i])
            continue
    if info_List[i][0] == "}":
        stack.append(info_List[i])

 
       
    
        


In [ ]:
info_List

In [ ]:
node_list

In [ ]:
class Node:
    def __init__(self, name):
        self.name = name
        self.offset = None
        self.channels = []
        self.children = []

    def __repr__(self):
        return f"Node({self.name})"


def build_skeleton(bvh_data):
    stack = []
    root = None
    current_node = None

    for line in bvh_data:
        if line[0] == 'ROOT' or line[0] == 'JOINT':
            # 创建新节点
            node = Node(line[1])
            if current_node is not None:
                # 添加为当前节点的子节点
                current_node.children.append(node)
                stack.append(current_node)
            else:
                root = node
            current_node = node
        elif line[0] == 'OFFSET':
            # 设置偏移量
            current_node.offset = tuple(map(float, line[1:]))
        elif line[0] == 'CHANNELS':
            # 设置通道信息
            current_node.channels = line[2:]  # 跳过通道数量
        elif line[0] == '}':
            # 结束当前节点的定义，返回到父节点
            if stack:
                current_node = stack.pop()
        elif line[0] == 'End' and line[1] == 'Site':
            # 处理末端节点
            end_site_node = Node('End Site')
            current_node.children.append(end_site_node)
            stack.append(current_node)
            current_node = end_site_node

    return root

# 解析示例数据
bvh_data = [['ROOT', 'Hips'], ['{'], ['OFFSET', '0.00', '0.00', '0.00'], ['CHANNELS', '6', 'Xposition', 'Yposition', 'Zposition', 'Xrotation', 'Yrotation', 'Zrotation']]
skeleton = build_skeleton(info_List)
print(skeleton.children)


In [ ]:
from dataclasses import dataclass
from collections import deque

@dataclass
class Node:
    name : str
    offset : list
    channels : dict
    child : list
    parent : str
    ID : int
    type : str
    
file_path = "walk60.bvh"
bvh_data = []
with open(file_path, 'r') as f:
    for line in f:
        bvh_data.append(line.split())

num_frames = 0
frame_time = 0
frames = []
joint_List = []
info_List = []
for line in bvh_data:
    this_line = line
    if(this_line[0] == "Hierarchy"): continue
    if(this_line[0] == "MOTION"): continue
    if(this_line[0] == "Frames:"):
        num_frames = int(this_line[1])
        continue
    if(this_line[0] == "Frame"):
        frame_time = float(this_line[2])
        continue
    if(this_line[0] == "{" or
       this_line[0] == "ROOT" or
       this_line[0] == "}" or
       this_line[0] == "End" or
       this_line[0] == "JOINT" or
       this_line[0] == "OFFSET" or
       this_line[0] == "CHANNELS"):
        info_List.append(this_line)
        continue
    frames.append(this_line)
    
offset_stack = deque()
channel_stack = deque()
node_stack = deque()
joint_List = []
node_id = 0

for info in info_List:
    if(info[0] == "ROOT"):
        node_stack.append(Node(
            name = "RootJoint",
            offset = None,
            channels = None,
            child = [],
            parent = None,
            ID = node_id,
            type = "root"
        ))
        node_id += 1
        continue
    elif info[0] == "{": continue
    elif info[0] == "OFFSET":
        offset_stack.append(list(map(float, info[1:])))
        continue
    elif info[0] == "CHANNELS":
        channel_stack.append({
            "channel_num" : int(info[1]),
            "channels" : info[2:]
        })
    elif info[0] == "JOINT":
        node_stack.append(Node(
            name = info[1],
            offset = None,
            channels = None,
            child = [],
            parent = None,
            ID = node_id,
            type = "joint"
        ))
        node_id += 1
        continue
    elif info[0] == "End":
        node_stack.append(Node(
            name = info[0] + info[1],
            offset = None,
            channels = None,
            parent = None,
            child = None,
            type = "End",
            ID = node_id
        ))
        node_id += 1
        continue
    elif info[0] == "}":
        pop_node = node_stack.pop()
        if pop_node.type == "End":
            top_node = node_stack[-1]
            pop_node.name = top_node.name + "_end"
            pop_node.parent = top_node.name
            pop_node.child = None
            offset_top = offset_stack.pop()
            pop_node.offset = offset_top
            pop_node.channels = None
            top_node.child.append(pop_node)
            joint_List.append(pop_node)
        elif pop_node.type == "joint":
            top_node = node_stack[-1]
            pop_offset = offset_stack.pop()
            pop_channels = channel_stack.pop()
            # print(pop_channels)
            pop_node.offset = pop_offset
            pop_node.channels = pop_channels
            pop_node.parent = top_node.name
            top_node.child.append(pop_node)
            joint_List.append(pop_node)
        elif pop_node.type == "root":
            pop_offset = offset_stack.pop()
            pop_channels = channel_stack.pop()
            # print(pop_channels)
            pop_node.offset = pop_offset
            pop_node.channels = pop_channels
            pop_node.parent = None
            joint_List.append(pop_node)
        continue
               
            

In [ ]:
for node in joint_List:
    if(node.name == "RootJoint"):
        print(len(node.child))

In [1]:
from read_bvh import Bone_Tree

bone_tree = Bone_Tree('walk60.bvh')

In [2]:
bone_tree.all_channels

[['Xposition',
  'Yposition',
  'Zposition',
  'Xrotation',
  'Yrotation',
  'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation'],
 ['Xrotation', 'Yrotation', 'Zrotation']]

In [5]:
bone_tree.channels

['Xposition',
 'Yposition',
 'Zposition',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation',
 'Xrotation',
 'Yrotation',
 'Zrotation']

In [6]:
bone_tree.frames[0]

['-0.001735',
 '0.855388',
 '0.315499',
 '2.008551',
 '7.606260',
 '-0.798294',
 '11.216058',
 '-3.286777',
 '-1.592436',
 '13.521250',
 '-1.153514',
 '-4.213484',
 '-17.754157',
 '-3.216621',
 '9.232892',
 '-7.948705',
 '0.211932',
 '-1.528529',
 '2.220789',
 '-0.981058',
 '-1.133630',
 '2.071938',
 '-6.311876',
 '2.083844',
 '2.020309',
 '-0.533885',
 '-19.342332',
 '-5.129554',
 '-37.575293',
 '-50.190804',
 '0.198025',
 '-24.741038',
 '4.442069',
 '0.442380',
 '2.547494',
 '4.858004',
 '1.951773',
 '-5.809334',
 '21.100535',
 '23.710456',
 '30.003467',
 '53.240376',
 '0.414981',
 '10.414544',
 '1.952633',
 '3.576914',
 '-9.482057',
 '6.918939',
 '1.457480',
 '-0.035296',
 '0.111891',
 '-27.722826',
 '-1.655032',
 '2.430426',
 '-2.964232',
 '-5.507982',
 '1.444119',
 '2.239212',
 '-3.180259',
 '-0.892285',
 '-0.008100',
 '-0.007000',
 '0.024400']

In [7]:
from read_bvh import Bone_Tree

In [12]:
import numpy as np
from scipy.spatial.transform import Rotation as R
def load_motion_data(bvh_file_path):
    """part2 辅助函数，读取bvh文件"""
    with open(bvh_file_path, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            if lines[i].startswith('Frame Time'):
                break
        motion_data = []
        for line in lines[i+1:]:
            data = [float(x) for x in line.split()]
            if len(data) == 0:
                break
            motion_data.append(np.array(data).reshape(1,-1))
        motion_data = np.concatenate(motion_data, axis=0)
    return motion_data

In [13]:
motion_data = load_motion_data(bvh_file_path= 'walk60.bvh')

In [14]:
motion_data

array([[-1.735000e-03,  8.553880e-01,  3.154990e-01, ..., -8.100000e-03,
        -7.000000e-03,  2.440000e-02],
       [-3.810000e-03,  8.539810e-01,  3.370020e-01, ..., -8.600000e-03,
        -7.000000e-03,  2.640000e-02],
       [-5.963000e-03,  8.528980e-01,  3.587130e-01, ..., -9.100000e-03,
        -7.100000e-03,  2.830000e-02],
       ...,
       [-2.948500e-02,  8.572510e-01,  3.906385e+00, ..., -5.951697e+00,
        -1.745390e-01,  1.760520e+00],
       [-2.775800e-02,  8.553910e-01,  3.926875e+00, ..., -7.948705e+00,
        -2.119320e-01,  1.528529e+00],
       [-2.563100e-02,  8.539840e-01,  3.948343e+00, ..., -9.397213e+00,
        -2.621580e-01,  1.565413e+00]])

In [16]:
print(motion_data.shape)

(182, 63)


In [17]:
motion_data_my = Bone_Tree('walk60.bvh').frames

In [18]:
motion_data_my

[['-0.001735',
  '0.855388',
  '0.315499',
  '2.008551',
  '7.606260',
  '-0.798294',
  '11.216058',
  '-3.286777',
  '-1.592436',
  '13.521250',
  '-1.153514',
  '-4.213484',
  '-17.754157',
  '-3.216621',
  '9.232892',
  '-7.948705',
  '0.211932',
  '-1.528529',
  '2.220789',
  '-0.981058',
  '-1.133630',
  '2.071938',
  '-6.311876',
  '2.083844',
  '2.020309',
  '-0.533885',
  '-19.342332',
  '-5.129554',
  '-37.575293',
  '-50.190804',
  '0.198025',
  '-24.741038',
  '4.442069',
  '0.442380',
  '2.547494',
  '4.858004',
  '1.951773',
  '-5.809334',
  '21.100535',
  '23.710456',
  '30.003467',
  '53.240376',
  '0.414981',
  '10.414544',
  '1.952633',
  '3.576914',
  '-9.482057',
  '6.918939',
  '1.457480',
  '-0.035296',
  '0.111891',
  '-27.722826',
  '-1.655032',
  '2.430426',
  '-2.964232',
  '-5.507982',
  '1.444119',
  '2.239212',
  '-3.180259',
  '-0.892285',
  '-0.008100',
  '-0.007000',
  '0.024400'],
 ['-0.003810',
  '0.853981',
  '0.337002',
  '2.017405',
  '7.825929',
  '

In [1]:
from read_bvh import Bone_Tree
tree = Bone_Tree(bvh_file_path= 'walk60.bvh')

In [2]:
tree.sorted_joint_list

[Node(name='RootJoint', offset=[0.0, 0.0, 0.0], channels={'channel_num': 6, 'channels': ['Xposition', 'Yposition', 'Zposition', 'Xrotation', 'Yrotation', 'Zrotation']}, child=[Node(name='lHip', offset=[0.1, -0.051395, 0.0], channels={'channel_num': 3, 'channels': ['Xrotation', 'Yrotation', 'Zrotation']}, child=[Node(name='lKnee', offset=[0.0, -0.41, 0.0], channels={'channel_num': 3, 'channels': ['Xrotation', 'Yrotation', 'Zrotation']}, child=[Node(name='lAnkle', offset=[0.0, -0.39, 0.0], channels={'channel_num': 3, 'channels': ['Xrotation', 'Yrotation', 'Zrotation']}, child=[Node(name='lToeJoint', offset=[0.0, -0.05, 0.13], channels={'channel_num': 3, 'channels': ['Xrotation', 'Yrotation', 'Zrotation']}, child=[Node(name='lToeJoint_end', offset=[0.01, 0.002, 0.06], channels=None, child=None, parent={'parent_name': 'lToeJoint', 'parent_id': 4}, ID=5, type='End')], parent={'parent_name': 'lAnkle', 'parent_id': 3}, ID=4, type='joint')], parent={'parent_name': 'lKnee', 'parent_id': 2}, ID=

In [3]:
name_list = []
id_list = []
parent_list = []
offset_list = []
for joint in tree.sorted_joint_list:
    name_list.append(joint.name)
    id_list.append(joint.ID)
    parent_list.append(joint.parent["parent_id"])
    offset_list.append(joint.offset)
print(name_list)
print(id_list)
print(parent_list)
print(len(offset_list))
print(tree.node_count)

['RootJoint', 'lHip', 'lKnee', 'lAnkle', 'lToeJoint', 'lToeJoint_end', 'pelvis_lowerback', 'lowerback_torso', 'lTorso_Clavicle', 'lShoulder', 'lElbow', 'lWrist', 'lWrist_end', 'rTorso_Clavicle', 'rShoulder', 'rElbow', 'rWrist', 'rWrist_end', 'torso_head', 'torso_head_end', 'rHip', 'rKnee', 'rAnkle', 'rToeJoint', 'rToeJoint_end']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[-1, 0, 1, 2, 3, 4, 0, 6, 7, 8, 9, 10, 11, 7, 13, 14, 15, 16, 7, 18, 0, 20, 21, 22, 23]
25
25


In [17]:
tree.joint_List

[Node(name='lToeJoint_end', offset=[0.01, 0.002, 0.06], channels=None, child=None, parent={'parent_name': 'lToeJoint', 'parent_id': 4}, ID=5, type='End'),
 Node(name='lToeJoint', offset=[0.0, -0.05, 0.13], channels={'channel_num': 3, 'channels': ['Xrotation', 'Yrotation', 'Zrotation']}, child=[Node(name='lToeJoint_end', offset=[0.01, 0.002, 0.06], channels=None, child=None, parent={'parent_name': 'lToeJoint', 'parent_id': 4}, ID=5, type='End')], parent={'parent_name': 'lAnkle', 'parent_id': 3}, ID=4, type='joint'),
 Node(name='lAnkle', offset=[0.0, -0.39, 0.0], channels={'channel_num': 3, 'channels': ['Xrotation', 'Yrotation', 'Zrotation']}, child=[Node(name='lToeJoint', offset=[0.0, -0.05, 0.13], channels={'channel_num': 3, 'channels': ['Xrotation', 'Yrotation', 'Zrotation']}, child=[Node(name='lToeJoint_end', offset=[0.01, 0.002, 0.06], channels=None, child=None, parent={'parent_name': 'lToeJoint', 'parent_id': 4}, ID=5, type='End')], parent={'parent_name': 'lAnkle', 'parent_id': 3},

In [2]:
from read_bvh import Bone_Tree
tree = Bone_Tree(bvh_file_path= 'walk60.bvh')

In [5]:
print(tree.joint_name)
print(tree.)
print(tree.joint_parent)

['RootJoint', 'lHip', 'lKnee', 'lAnkle', 'lToeJoint', 'lToeJoint_end', 'pelvis_lowerback', 'lowerback_torso', 'lTorso_Clavicle', 'lShoulder', 'lElbow', 'lWrist', 'lWrist_end', 'rTorso_Clavicle', 'rShoulder', 'rElbow', 'rWrist', 'rWrist_end', 'torso_head', 'torso_head_end', 'rHip', 'rKnee', 'rAnkle', 'rToeJoint', 'rToeJoint_end']
[[ 0.        0.        0.      ]
 [ 0.1      -0.051395  0.      ]
 [ 0.       -0.41      0.      ]
 [ 0.       -0.39      0.      ]
 [ 0.       -0.05      0.13    ]
 [ 0.01      0.002     0.06    ]
 [ 0.        0.093605  0.      ]
 [ 0.        0.1       0.      ]
 [ 0.001     0.1575    0.      ]
 [ 0.117647  0.        0.      ]
 [ 0.245     0.        0.      ]
 [ 0.24      0.        0.      ]
 [ 0.116353 -0.0025    0.      ]
 [-0.001     0.1575    0.      ]
 [-0.117647  0.        0.      ]
 [-0.245     0.        0.      ]
 [-0.24      0.        0.      ]
 [-0.116353 -0.0025    0.      ]
 [ 0.        0.28235   0.      ]
 [ 0.        0.19265   0.      ]
 [-0.1   